In [2]:
import pandas as pd
import numpy as np
import os
import joblib
from difflib import SequenceMatcher
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart
from openpyxl.chart.reference import Reference


# 1. load saved models and preprocessing objects
model_dir = 'saved_models_additional_v2'
models = {
    'illness': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_illness.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_illness.pkl'))
    },
    'injury': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_injury.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_injury.pkl'))
    }
}

# load vectorizers and encoders
vectorizers = {
    'illness': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_illness.pkl')),
    'injury': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_injury.pkl'))
}

label_encoders = {
    'illness': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_illness_label_encoder.pkl'))
    },
    'injury': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_injury_label_encoder.pkl'))
    }
}

# 2. load and prepare data with column name standardization
def load_and_prepare_data(filepath):
    df = pd.read_excel(filepath)
    
    # standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # handle missing values
    df['illness_information'] = df['illness_information'].fillna("No Illness")
    df['injury_information'] = df['injury_information'].fillna("No Injury")
    
    # ensure required columns exist
    if 'user_id' not in df.columns:
        df['user_id'] = np.arange(1, len(df)+1)
    
    if 'date' not in df.columns:
        df['date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # convert date to string if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

test_df = load_and_prepare_data("cleaned_test_data_v3.xlsx")
realuse_df = load_and_prepare_data("cleaned_realuse_data_v3.xlsx")

# 3. enhanced prediction function
def predict_with_models(text, problem_type, actual):
    predictions = {}
    if text in ["No Illness", "No Injury"]:
        return {model: ("No Prediction", 0) for model in models[problem_type].keys()}
    
    vectorized = vectorizers[problem_type].transform([text])
    
    for model_name, model in models[problem_type].items():
        pred = model.predict(vectorized)[0]
        
        if model_name in ['Logistic Regression', 'XGBoost']:
            pred = label_encoders[problem_type][model_name].inverse_transform([pred])[0]
        
        similarity = SequenceMatcher(None, str(actual), str(pred)).ratio() * 100
        predictions[model_name] = (pred, similarity)
    
    return predictions

# 4. process dataset to create flattened results - only for rows with illness or injury
def process_dataset(df):
    illness_results = []
    injury_results = []
    
    for _, row in df.iterrows():
        user_id = str(row['user_id'])
        date = str(row['date'])
        
        # process illness if present
        if row['illed'] == 1 and row['illness_information'] != "No Illness":
            illness_actual = row['illness_information']
            illness_preds = predict_with_models(illness_actual, 'illness', illness_actual)
            
            illness_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': illness_actual
            }
            
            for model_name, (predicted, similarity) in illness_preds.items():
                illness_data[f"{model_name} Predicted"] = predicted
                illness_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                illness_data[f"{model_name} Correct"] = illness_actual == predicted if predicted != "No Prediction" else False
            
            illness_results.append(illness_data)
        
        # process injury if present
        if row['injured'] == 1 and row['injury_information'] != "No Injury":
            injury_actual = row['injury_information']
            injury_preds = predict_with_models(injury_actual, 'injury', injury_actual)
            
            injury_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': injury_actual
            }
            
            for model_name, (predicted, similarity) in injury_preds.items():
                injury_data[f"{model_name} Predicted"] = predicted
                injury_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                injury_data[f"{model_name} Correct"] = injury_actual == predicted if predicted != "No Prediction" else False
            
            injury_results.append(injury_data)
            
    def sort_dataframe(df):
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(['Date', 'User ID'])
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df
    
    return {
        'illness': sort_dataframe(pd.DataFrame(illness_results)),
        'injury': sort_dataframe(pd.DataFrame(injury_results))
    }

# 5. process both datasets
test_results = {
    'illness': process_dataset(test_df)['illness'],
    'injury': process_dataset(test_df)['injury']
}

realuse_results = {
    'illness': process_dataset(realuse_df)['illness'],
    'injury': process_dataset(realuse_df)['injury']
}

# 6. create Excel report with enhanced formatting
def create_excel_report():
    output_file = 'additional_report_no_severity.xlsx'
    print(f"\nGenerating Excel report: {output_file}")
    
    # create styles
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    header_font = Font(color='FFFFFF', bold=True)
    summary_header_fill = PatternFill(start_color='7030A0', end_color='7030A0', fill_type='solid')
    center_alignment = Alignment(horizontal='center')
    
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',
        mid_type='num', mid_value=50, mid_color='FFFF00',
        end_type='num', end_value=100, end_color='00FF00'
    )

    try:
        # create a new workbook
        wb = openpyxl.Workbook()
        
        # create sheets for each dataset and problem type
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                sheet_name = f"{dataset_name} {problem_type.capitalize()}"[:31]
                ws = wb.create_sheet(sheet_name)
                
                # write the data to the worksheet
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
                    for c_idx, value in enumerate(row, 1):
                        ws.cell(row=r_idx, column=c_idx, value=value)
                
                # apply header formatting
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = center_alignment
                
                # find accuracy and correct columns
                for col_idx, col in enumerate(ws.iter_cols(), 1):
                    col_name = col[0].value
                    
                    if col_name and 'Accuracy (%)' in col_name:
                        # apply color scale to accuracy columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            color_scale_rule
                        )
                    
                    elif col_name and 'Correct' in col_name:
                        # apply green/red to correctness columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['TRUE'], fill=green_fill)
                        )
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['FALSE'], fill=red_fill)
                        )
                
                # auto-size columns
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    adjusted_width = (max_length + 2)
                    ws.column_dimensions[column_letter].width = adjusted_width
        
        # create summary sheet
        ws_summary = wb.create_sheet("Model Performance Summary")
        
        # prepare summary data
        summary_data = []
        models_list = ['Random Forest', 'Logistic Regression', 'XGBoost']
        
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                for model in models_list:
                    # calculate average accuracy
                    acc_col = f"{model} Accuracy (%)"
                    if acc_col in df.columns:
                        avg_accuracy = df[acc_col].mean()
                        
                        # calculate error rate (false predictions percentage)
                        correct_col = f"{model} Correct"
                        if correct_col in df.columns:
                            total_predictions = len(df[correct_col])
                            true_predictions = len(df[df[correct_col] == True])
                            accuracy_rate = (true_predictions / total_predictions) * 100 if total_predictions > 0 else 0
                            
                            summary_data.append({
                                'Dataset': dataset_name,
                                'Problem Type': problem_type.capitalize(),
                                'Model': model,
                                'Average Accuracy (%)': round(avg_accuracy, 1),
                                'Accuracy Rate (%)': round(accuracy_rate, 1),
                                'Total Predictions': total_predictions
                            })
        
        # create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        
        # write summary to Excel
        for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws_summary.cell(row=r_idx, column=c_idx, value=value)
        
        # format summary sheet
        for cell in ws_summary[1]:
            cell.fill = summary_header_fill
            cell.font = header_font
            cell.alignment = center_alignment
        
        # apply conditional formatting to accuracy and error rate
        ws_summary.conditional_formatting.add(
            f"D2:D{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000',
                mid_type='num', mid_value=50, mid_color='FFFF00',
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        ws_summary.conditional_formatting.add(
            f"E2:E{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000', 
                mid_type='num', mid_value=50, mid_color='FFFF00', 
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        # Auto-size columns
        for column in ws_summary.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_summary.column_dimensions[column_letter].width = adjusted_width
        
        # Add charts to summary sheet
        chart1 = BarChart()
        chart1.title = "Average Accuracy by Model"
        chart1.y_axis.title = "Accuracy (%)"
        chart1.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=4, min_row=1, max_row=ws_summary.max_row, max_col=4)
        cats = Reference(ws_summary, min_col=3, min_row=2, max_row=ws_summary.max_row)
        chart1.add_data(data, titles_from_data=True)
        chart1.set_categories(cats)
        ws_summary.add_chart(chart1, "G2")
        
        chart2 = BarChart()
        chart2.title = "Error Rate by Model"
        chart2.y_axis.title = "Error Rate (%)"
        chart2.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=5, min_row=1, max_row=ws_summary.max_row, max_col=5)
        chart2.add_data(data, titles_from_data=True)
        chart2.set_categories(cats)
        ws_summary.add_chart(chart2, "G20")
        
        # Remove default sheet if it exists
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # Save the workbook
        wb.save(output_file)
        print(f"Successfully saved report to: {os.path.abspath(output_file)}")
        return output_file
    
    except Exception as e:
        print(f"Error generating Excel report: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 7. generate the report
report_file = create_excel_report()



Generating Excel report: additional_report_no_severity.xlsx
Successfully saved report to: /Users/dingyangzuo/additional_report_no_severity.xlsx


In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from difflib import SequenceMatcher
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart
from openpyxl.chart.reference import Reference


# 1. load saved models and preprocessing objects
model_dir = 'saved_models_additional_training'
models = {
    'illness': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_illness.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_illness.pkl'))
    },
    'injury': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_injury.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_injury.pkl'))
    }
}

# load vectorizers and encoders
vectorizers = {
    'illness': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_illness.pkl')),
    'injury': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_injury.pkl'))
}

label_encoders = {
    'illness': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_illness_label_encoder.pkl'))
    },
    'injury': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_injury_label_encoder.pkl'))
    }
}

# 2. load and prepare data with column name standardization
def load_and_prepare_data(filepath):
    df = pd.read_excel(filepath)
    
    # standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # handle missing values
    df['illness_information'] = df['illness_information'].fillna("No Illness")
    df['injury_information'] = df['injury_information'].fillna("No Injury")
    
    # ensure required columns exist
    if 'user_id' not in df.columns:
        df['user_id'] = np.arange(1, len(df)+1)
    
    if 'date' not in df.columns:
        df['date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # convert date to string if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

test_df = load_and_prepare_data("cleaned_test_data_v3.xlsx")
realuse_df = load_and_prepare_data("cleaned_realuse_data_v3.xlsx")

# 3. enhanced prediction function
def predict_with_models(text, problem_type, actual):
    predictions = {}
    if text in ["No Illness", "No Injury"]:
        return {model: ("No Prediction", 0) for model in models[problem_type].keys()}
    
    vectorized = vectorizers[problem_type].transform([text])
    
    for model_name, model in models[problem_type].items():
        pred = model.predict(vectorized)[0]
        
        if model_name in ['Logistic Regression', 'XGBoost']:
            pred = label_encoders[problem_type][model_name].inverse_transform([pred])[0]
        
        similarity = SequenceMatcher(None, str(actual), str(pred)).ratio() * 100
        predictions[model_name] = (pred, similarity)
    
    return predictions

# 4. process dataset to create flattened results - only for rows with illness or injury
def process_dataset(df):
    illness_results = []
    injury_results = []
    
    for _, row in df.iterrows():
        user_id = str(row['user_id'])
        date = str(row['date'])
        
        # process illness if present
        if row['illed'] == 1 and row['illness_information'] != "No Illness":
            illness_actual = row['illness_information']
            illness_preds = predict_with_models(illness_actual, 'illness', illness_actual)
            
            illness_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': illness_actual
            }
            
            for model_name, (predicted, similarity) in illness_preds.items():
                illness_data[f"{model_name} Predicted"] = predicted
                illness_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                illness_data[f"{model_name} Correct"] = illness_actual == predicted if predicted != "No Prediction" else False
            
            illness_results.append(illness_data)
        
        # process injury if present
        if row['injured'] == 1 and row['injury_information'] != "No Injury":
            injury_actual = row['injury_information']
            injury_preds = predict_with_models(injury_actual, 'injury', injury_actual)
            
            injury_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': injury_actual
            }
            
            for model_name, (predicted, similarity) in injury_preds.items():
                injury_data[f"{model_name} Predicted"] = predicted
                injury_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                injury_data[f"{model_name} Correct"] = injury_actual == predicted if predicted != "No Prediction" else False
            
            injury_results.append(injury_data)
            
    def sort_dataframe(df):
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(['Date', 'User ID'])
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df
    
    return {
        'illness': sort_dataframe(pd.DataFrame(illness_results)),
        'injury': sort_dataframe(pd.DataFrame(injury_results))
    }

# 5. process both datasets
test_results = {
    'illness': process_dataset(test_df)['illness'],
    'injury': process_dataset(test_df)['injury']
}

realuse_results = {
    'illness': process_dataset(realuse_df)['illness'],
    'injury': process_dataset(realuse_df)['injury']
}

# 6. create Excel report with enhanced formatting
def create_excel_report():
    output_file = 'additional_report_v2.xlsx'
    print(f"\nGenerating Excel report: {output_file}")
    
    # create styles
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    header_font = Font(color='FFFFFF', bold=True)
    summary_header_fill = PatternFill(start_color='7030A0', end_color='7030A0', fill_type='solid')
    center_alignment = Alignment(horizontal='center')
    
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',
        mid_type='num', mid_value=50, mid_color='FFFF00',
        end_type='num', end_value=100, end_color='00FF00'
    )

    try:
        # create a new workbook
        wb = openpyxl.Workbook()
        
        # create sheets for each dataset and problem type
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                sheet_name = f"{dataset_name} {problem_type.capitalize()}"[:31]
                ws = wb.create_sheet(sheet_name)
                
                # write the data to the worksheet
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
                    for c_idx, value in enumerate(row, 1):
                        ws.cell(row=r_idx, column=c_idx, value=value)
                
                # apply header formatting
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = center_alignment
                
                # find accuracy and correct columns
                for col_idx, col in enumerate(ws.iter_cols(), 1):
                    col_name = col[0].value
                    
                    if col_name and 'Accuracy (%)' in col_name:
                        # apply color scale to accuracy columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            color_scale_rule
                        )
                    
                    elif col_name and 'Correct' in col_name:
                        # apply green/red to correctness columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['TRUE'], fill=green_fill)
                        )
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['FALSE'], fill=red_fill)
                        )
                
                # auto-size columns
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    adjusted_width = (max_length + 2)
                    ws.column_dimensions[column_letter].width = adjusted_width
        
        # create summary sheet
        ws_summary = wb.create_sheet("Model Performance Summary")
        
        # prepare summary data
        summary_data = []
        models_list = ['Random Forest', 'Logistic Regression', 'XGBoost']
        
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                for model in models_list:
                    # calculate average accuracy
                    acc_col = f"{model} Accuracy (%)"
                    if acc_col in df.columns:
                        avg_accuracy = df[acc_col].mean()
                        
                        # calculate error rate (false predictions percentage)
                        correct_col = f"{model} Correct"
                        if correct_col in df.columns:
                            total_predictions = len(df[correct_col])
                            true_predictions = len(df[df[correct_col] == True])
                            accuracy_rate = (true_predictions / total_predictions) * 100 if total_predictions > 0 else 0
                            
                            summary_data.append({
                                'Dataset': dataset_name,
                                'Problem Type': problem_type.capitalize(),
                                'Model': model,
                                'Average Accuracy (%)': round(avg_accuracy, 1),
                                'Accuracy Rate (%)': round(accuracy_rate, 1),
                                'Total Predictions': total_predictions
                            })
        
        # create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        
        # write summary to Excel
        for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws_summary.cell(row=r_idx, column=c_idx, value=value)
        
        # format summary sheet
        for cell in ws_summary[1]:
            cell.fill = summary_header_fill
            cell.font = header_font
            cell.alignment = center_alignment
        
        # apply conditional formatting to accuracy and error rate
        ws_summary.conditional_formatting.add(
            f"D2:D{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000',
                mid_type='num', mid_value=50, mid_color='FFFF00',
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        ws_summary.conditional_formatting.add(
            f"E2:E{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000', 
                mid_type='num', mid_value=50, mid_color='FFFF00', 
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        # Auto-size columns
        for column in ws_summary.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_summary.column_dimensions[column_letter].width = adjusted_width
        
        # Add charts to summary sheet
        chart1 = BarChart()
        chart1.title = "Average Accuracy by Model"
        chart1.y_axis.title = "Accuracy (%)"
        chart1.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=4, min_row=1, max_row=ws_summary.max_row, max_col=4)
        cats = Reference(ws_summary, min_col=3, min_row=2, max_row=ws_summary.max_row)
        chart1.add_data(data, titles_from_data=True)
        chart1.set_categories(cats)
        ws_summary.add_chart(chart1, "G2")
        
        chart2 = BarChart()
        chart2.title = "Error Rate by Model"
        chart2.y_axis.title = "Error Rate (%)"
        chart2.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=5, min_row=1, max_row=ws_summary.max_row, max_col=5)
        chart2.add_data(data, titles_from_data=True)
        chart2.set_categories(cats)
        ws_summary.add_chart(chart2, "G20")
        
        # Remove default sheet if it exists
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # Save the workbook
        wb.save(output_file)
        print(f"Successfully saved report to: {os.path.abspath(output_file)}")
        return output_file
    
    except Exception as e:
        print(f"Error generating Excel report: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 7. generate the report
report_file = create_excel_report()



Generating Excel report: additional_report_v2.xlsx
Successfully saved report to: /Users/dingyangzuo/additional_report_v2.xlsx


In [3]:
import pandas as pd
import numpy as np
import os
import joblib
from difflib import SequenceMatcher
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart
from openpyxl.chart.reference import Reference


# 1. load saved models and preprocessing objects
model_dir = 'saved_models_additional_v2_no_severity'
models = {
    'illness': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_illness.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_illness.pkl'))
    },
    'injury': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_injury.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_injury.pkl'))
    }
}

# load vectorizers and encoders
vectorizers = {
    'illness': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_illness.pkl')),
    'injury': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_injury.pkl'))
}

label_encoders = {
    'illness': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_illness_label_encoder.pkl'))
    },
    'injury': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_injury_label_encoder.pkl'))
    }
}

# 2. load and prepare data with column name standardization
def load_and_prepare_data(filepath):
    df = pd.read_excel(filepath)
    
    # standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # handle missing values
    df['illness_information'] = df['illness_information'].fillna("No Illness")
    df['injury_information'] = df['injury_information'].fillna("No Injury")
    
    # ensure required columns exist
    if 'user_id' not in df.columns:
        df['user_id'] = np.arange(1, len(df)+1)
    
    if 'date' not in df.columns:
        df['date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # convert date to string if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

test_df = load_and_prepare_data("cleaned_test_data_v3.xlsx")
realuse_df = load_and_prepare_data("cleaned_realuse_data_v3.xlsx")

# 3. enhanced prediction function
def predict_with_models(text, problem_type, actual):
    predictions = {}
    if text in ["No Illness", "No Injury"]:
        return {model: ("No Prediction", 0) for model in models[problem_type].keys()}
    
    vectorized = vectorizers[problem_type].transform([text])
    
    for model_name, model in models[problem_type].items():
        pred = model.predict(vectorized)[0]
        
        if model_name in ['Logistic Regression', 'XGBoost']:
            pred = label_encoders[problem_type][model_name].inverse_transform([pred])[0]
        
        similarity = SequenceMatcher(None, str(actual), str(pred)).ratio() * 100
        predictions[model_name] = (pred, similarity)
    
    return predictions

# 4. process dataset to create flattened results - only for rows with illness or injury
def process_dataset(df):
    illness_results = []
    injury_results = []
    
    for _, row in df.iterrows():
        user_id = str(row['user_id'])
        date = str(row['date'])
        
        # process illness if present
        if row['illed'] == 1 and row['illness_information'] != "No Illness":
            illness_actual = row['illness_information']
            illness_preds = predict_with_models(illness_actual, 'illness', illness_actual)
            
            illness_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': illness_actual
            }
            
            for model_name, (predicted, similarity) in illness_preds.items():
                illness_data[f"{model_name} Predicted"] = predicted
                illness_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                illness_data[f"{model_name} Correct"] = illness_actual == predicted if predicted != "No Prediction" else False
            
            illness_results.append(illness_data)
        
        # process injury if present
        if row['injured'] == 1 and row['injury_information'] != "No Injury":
            injury_actual = row['injury_information']
            injury_preds = predict_with_models(injury_actual, 'injury', injury_actual)
            
            injury_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': injury_actual
            }
            
            for model_name, (predicted, similarity) in injury_preds.items():
                injury_data[f"{model_name} Predicted"] = predicted
                injury_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                injury_data[f"{model_name} Correct"] = injury_actual == predicted if predicted != "No Prediction" else False
            
            injury_results.append(injury_data)
            
    def sort_dataframe(df):
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(['Date', 'User ID'])
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df
    
    return {
        'illness': sort_dataframe(pd.DataFrame(illness_results)),
        'injury': sort_dataframe(pd.DataFrame(injury_results))
    }

# 5. process both datasets
test_results = {
    'illness': process_dataset(test_df)['illness'],
    'injury': process_dataset(test_df)['injury']
}

realuse_results = {
    'illness': process_dataset(realuse_df)['illness'],
    'injury': process_dataset(realuse_df)['injury']
}

# 6. create Excel report with enhanced formatting
def create_excel_report():
    output_file = 'additional_report_no_severity_v2.xlsx'
    print(f"\nGenerating Excel report: {output_file}")
    
    # create styles
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    header_font = Font(color='FFFFFF', bold=True)
    summary_header_fill = PatternFill(start_color='7030A0', end_color='7030A0', fill_type='solid')
    center_alignment = Alignment(horizontal='center')
    
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',
        mid_type='num', mid_value=50, mid_color='FFFF00',
        end_type='num', end_value=100, end_color='00FF00'
    )

    try:
        # create a new workbook
        wb = openpyxl.Workbook()
        
        # create sheets for each dataset and problem type
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                sheet_name = f"{dataset_name} {problem_type.capitalize()}"[:31]
                ws = wb.create_sheet(sheet_name)
                
                # write the data to the worksheet
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
                    for c_idx, value in enumerate(row, 1):
                        ws.cell(row=r_idx, column=c_idx, value=value)
                
                # apply header formatting
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = center_alignment
                
                # find accuracy and correct columns
                for col_idx, col in enumerate(ws.iter_cols(), 1):
                    col_name = col[0].value
                    
                    if col_name and 'Accuracy (%)' in col_name:
                        # apply color scale to accuracy columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            color_scale_rule
                        )
                    
                    elif col_name and 'Correct' in col_name:
                        # apply green/red to correctness columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['TRUE'], fill=green_fill)
                        )
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['FALSE'], fill=red_fill)
                        )
                
                # auto-size columns
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    adjusted_width = (max_length + 2)
                    ws.column_dimensions[column_letter].width = adjusted_width
        
        # create summary sheet
        ws_summary = wb.create_sheet("Model Performance Summary")
        
        # prepare summary data
        summary_data = []
        models_list = ['Random Forest', 'Logistic Regression', 'XGBoost']
        
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                for model in models_list:
                    # calculate average accuracy
                    acc_col = f"{model} Accuracy (%)"
                    if acc_col in df.columns:
                        avg_accuracy = df[acc_col].mean()
                        
                        # calculate error rate (false predictions percentage)
                        correct_col = f"{model} Correct"
                        if correct_col in df.columns:
                            total_predictions = len(df[correct_col])
                            true_predictions = len(df[df[correct_col] == True])
                            accuracy_rate = (true_predictions / total_predictions) * 100 if total_predictions > 0 else 0
                            
                            summary_data.append({
                                'Dataset': dataset_name,
                                'Problem Type': problem_type.capitalize(),
                                'Model': model,
                                'Average Accuracy (%)': round(avg_accuracy, 1),
                                'Accuracy Rate (%)': round(accuracy_rate, 1),
                                'Total Predictions': total_predictions
                            })
        
        # create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        
        # write summary to Excel
        for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws_summary.cell(row=r_idx, column=c_idx, value=value)
        
        # format summary sheet
        for cell in ws_summary[1]:
            cell.fill = summary_header_fill
            cell.font = header_font
            cell.alignment = center_alignment
        
        # apply conditional formatting to accuracy and error rate
        ws_summary.conditional_formatting.add(
            f"D2:D{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000',
                mid_type='num', mid_value=50, mid_color='FFFF00',
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        ws_summary.conditional_formatting.add(
            f"E2:E{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000', 
                mid_type='num', mid_value=50, mid_color='FFFF00', 
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        # Auto-size columns
        for column in ws_summary.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_summary.column_dimensions[column_letter].width = adjusted_width
        
        # Add charts to summary sheet
        chart1 = BarChart()
        chart1.title = "Average Accuracy by Model"
        chart1.y_axis.title = "Accuracy (%)"
        chart1.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=4, min_row=1, max_row=ws_summary.max_row, max_col=4)
        cats = Reference(ws_summary, min_col=3, min_row=2, max_row=ws_summary.max_row)
        chart1.add_data(data, titles_from_data=True)
        chart1.set_categories(cats)
        ws_summary.add_chart(chart1, "G2")
        
        chart2 = BarChart()
        chart2.title = "Error Rate by Model"
        chart2.y_axis.title = "Error Rate (%)"
        chart2.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=5, min_row=1, max_row=ws_summary.max_row, max_col=5)
        chart2.add_data(data, titles_from_data=True)
        chart2.set_categories(cats)
        ws_summary.add_chart(chart2, "G20")
        
        # Remove default sheet if it exists
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # Save the workbook
        wb.save(output_file)
        print(f"Successfully saved report to: {os.path.abspath(output_file)}")
        return output_file
    
    except Exception as e:
        print(f"Error generating Excel report: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 7. generate the report
report_file = create_excel_report()



Generating Excel report: additional_report_no_severity_v2.xlsx
Successfully saved report to: /Users/dingyangzuo/additional_report_no_severity_v2.xlsx


In [4]:
import pandas as pd
import numpy as np
import os
import joblib
from difflib import SequenceMatcher
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart
from openpyxl.chart.reference import Reference


# 1. load saved models and preprocessing objects
model_dir = 'saved_models_additional_v3'
models = {
    'illness': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_illness.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_illness.pkl'))
    },
    'injury': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_injury.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_injury.pkl'))
    }
}

# load vectorizers and encoders
vectorizers = {
    'illness': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_illness.pkl')),
    'injury': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_injury.pkl'))
}

label_encoders = {
    'illness': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_illness_label_encoder.pkl'))
    },
    'injury': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_injury_label_encoder.pkl'))
    }
}

# 2. load and prepare data with column name standardization
def load_and_prepare_data(filepath):
    df = pd.read_excel(filepath)
    
    # standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    illness_columns = ['type_of_illness', 'illness_severity']
    injury_columns = ['injury_location', 'injury_type', 'injury_severity']
    df['illness_text'] = df[illness_columns].fillna('').agg(' '.join, axis=1)
    df['injury_text'] = df[injury_columns].fillna('').agg(' '.join, axis=1)
    df['injury_information'] = df['injury_text'].fillna("No Injury")
    df['illness_information'] = df['illness_text'].fillna("No Illness")
    
    # ensure required columns exist
    if 'user_id' not in df.columns:
        df['user_id'] = np.arange(1, len(df)+1)
    
    if 'date' not in df.columns:
        df['date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # convert date to string if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

test_df = load_and_prepare_data("cleaned_test_data_v3.xlsx")
realuse_df = load_and_prepare_data("cleaned_realuse_data_v3.xlsx")

# 3. enhanced prediction function
def predict_with_models(text, problem_type, actual):
    predictions = {}
    if text in ["No Illness", "No Injury"]:
        return {model: ("No Prediction", 0) for model in models[problem_type].keys()}
    
    vectorized = vectorizers[problem_type].transform([text])
    
    for model_name, model in models[problem_type].items():
        pred = model.predict(vectorized)[0]
        
        if model_name in ['Logistic Regression', 'XGBoost']:
            pred = label_encoders[problem_type][model_name].inverse_transform([pred])[0]
        
        similarity = SequenceMatcher(None, str(actual), str(pred)).ratio() * 100
        predictions[model_name] = (pred, similarity)
    
    return predictions

# 4. process dataset to create flattened results - only for rows with illness or injury
def process_dataset(df):
    illness_results = []
    injury_results = []
    
    for _, row in df.iterrows():
        user_id = str(row['user_id'])
        date = str(row['date'])
        
        # process illness if present
        if row['illed'] == 1 and row['illness_information'] != "No Illness":
            illness_actual = row['illness_information']
            illness_preds = predict_with_models(illness_actual, 'illness', illness_actual)
            
            illness_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': illness_actual
            }
            
            for model_name, (predicted, similarity) in illness_preds.items():
                illness_data[f"{model_name} Predicted"] = predicted
                illness_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                illness_data[f"{model_name} Correct"] = illness_actual == predicted if predicted != "No Prediction" else False
            
            illness_results.append(illness_data)
        
        # process injury if present
        if row['injured'] == 1 and row['injury_information'] != "No Injury":
            injury_actual = row['injury_information']
            injury_preds = predict_with_models(injury_actual, 'injury', injury_actual)
            
            injury_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': injury_actual
            }
            
            for model_name, (predicted, similarity) in injury_preds.items():
                injury_data[f"{model_name} Predicted"] = predicted
                injury_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                injury_data[f"{model_name} Correct"] = injury_actual == predicted if predicted != "No Prediction" else False
            
            injury_results.append(injury_data)
            
    def sort_dataframe(df):
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(['Date', 'User ID'])
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df
    
    return {
        'illness': sort_dataframe(pd.DataFrame(illness_results)),
        'injury': sort_dataframe(pd.DataFrame(injury_results))
    }

# 5. process both datasets
test_results = {
    'illness': process_dataset(test_df)['illness'],
    'injury': process_dataset(test_df)['injury']
}

realuse_results = {
    'illness': process_dataset(realuse_df)['illness'],
    'injury': process_dataset(realuse_df)['injury']
}

# 6. create Excel report with enhanced formatting
def create_excel_report():
    output_file = 'additional_report_v3.xlsx'
    print(f"\nGenerating Excel report: {output_file}")
    
    # create styles
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    header_font = Font(color='FFFFFF', bold=True)
    summary_header_fill = PatternFill(start_color='7030A0', end_color='7030A0', fill_type='solid')
    center_alignment = Alignment(horizontal='center')
    
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',
        mid_type='num', mid_value=50, mid_color='FFFF00',
        end_type='num', end_value=100, end_color='00FF00'
    )

    try:
        # create a new workbook
        wb = openpyxl.Workbook()
        
        # create sheets for each dataset and problem type
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                sheet_name = f"{dataset_name} {problem_type.capitalize()}"[:31]
                ws = wb.create_sheet(sheet_name)
                
                # write the data to the worksheet
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
                    for c_idx, value in enumerate(row, 1):
                        ws.cell(row=r_idx, column=c_idx, value=value)
                
                # apply header formatting
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = center_alignment
                
                # find accuracy and correct columns
                for col_idx, col in enumerate(ws.iter_cols(), 1):
                    col_name = col[0].value
                    
                    if col_name and 'Accuracy (%)' in col_name:
                        # apply color scale to accuracy columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            color_scale_rule
                        )
                    
                    elif col_name and 'Correct' in col_name:
                        # apply green/red to correctness columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['TRUE'], fill=green_fill)
                        )
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['FALSE'], fill=red_fill)
                        )
                
                # auto-size columns
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    adjusted_width = (max_length + 2)
                    ws.column_dimensions[column_letter].width = adjusted_width
        
        # create summary sheet
        ws_summary = wb.create_sheet("Model Performance Summary")
        
        # prepare summary data
        summary_data = []
        models_list = ['Random Forest', 'Logistic Regression', 'XGBoost']
        
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                for model in models_list:
                    # calculate average accuracy
                    acc_col = f"{model} Accuracy (%)"
                    if acc_col in df.columns:
                        avg_accuracy = df[acc_col].mean()
                        
                        # calculate error rate (false predictions percentage)
                        correct_col = f"{model} Correct"
                        if correct_col in df.columns:
                            total_predictions = len(df[correct_col])
                            true_predictions = len(df[df[correct_col] == True])
                            accuracy_rate = (true_predictions / total_predictions) * 100 if total_predictions > 0 else 0
                            
                            summary_data.append({
                                'Dataset': dataset_name,
                                'Problem Type': problem_type.capitalize(),
                                'Model': model,
                                'Average Accuracy (%)': round(avg_accuracy, 1),
                                'Accuracy Rate (%)': round(accuracy_rate, 1),
                                'Total Predictions': total_predictions
                            })
        
        # create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        
        # write summary to Excel
        for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws_summary.cell(row=r_idx, column=c_idx, value=value)
        
        # format summary sheet
        for cell in ws_summary[1]:
            cell.fill = summary_header_fill
            cell.font = header_font
            cell.alignment = center_alignment
        
        # apply conditional formatting to accuracy and error rate
        ws_summary.conditional_formatting.add(
            f"D2:D{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000',
                mid_type='num', mid_value=50, mid_color='FFFF00',
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        ws_summary.conditional_formatting.add(
            f"E2:E{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000', 
                mid_type='num', mid_value=50, mid_color='FFFF00', 
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        # Auto-size columns
        for column in ws_summary.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_summary.column_dimensions[column_letter].width = adjusted_width
        
        # Add charts to summary sheet
        chart1 = BarChart()
        chart1.title = "Average Accuracy by Model"
        chart1.y_axis.title = "Accuracy (%)"
        chart1.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=4, min_row=1, max_row=ws_summary.max_row, max_col=4)
        cats = Reference(ws_summary, min_col=3, min_row=2, max_row=ws_summary.max_row)
        chart1.add_data(data, titles_from_data=True)
        chart1.set_categories(cats)
        ws_summary.add_chart(chart1, "G2")
        
        chart2 = BarChart()
        chart2.title = "Error Rate by Model"
        chart2.y_axis.title = "Error Rate (%)"
        chart2.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=5, min_row=1, max_row=ws_summary.max_row, max_col=5)
        chart2.add_data(data, titles_from_data=True)
        chart2.set_categories(cats)
        ws_summary.add_chart(chart2, "G20")
        
        # Remove default sheet if it exists
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # Save the workbook
        wb.save(output_file)
        print(f"Successfully saved report to: {os.path.abspath(output_file)}")
        return output_file
    
    except Exception as e:
        print(f"Error generating Excel report: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 7. generate the report
report_file = create_excel_report()



Generating Excel report: additional_report_v3.xlsx
Successfully saved report to: /Users/dingyangzuo/additional_report_v3.xlsx


In [6]:
import pandas as pd
import numpy as np
import os
import joblib
from difflib import SequenceMatcher
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart
from openpyxl.chart.reference import Reference


# 1. load saved models and preprocessing objects
model_dir = 'saved_models_additional_v2_no_severity'
models = {
    'illness': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_illness.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_illness.pkl'))
    },
    'injury': {
        'Random Forest': joblib.load(os.path.join(model_dir, 'rf_model_injury.pkl')),
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'log_reg_model_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_model_injury.pkl'))
    }
}

# load vectorizers and encoders
vectorizers = {
    'illness': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_illness.pkl')),
    'injury': joblib.load(os.path.join(model_dir, 'tfidf_vectorizer_injury.pkl'))
}

label_encoders = {
    'illness': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_illness.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_illness_label_encoder.pkl'))
    },
    'injury': {
        'Logistic Regression': joblib.load(os.path.join(model_dir, 'label_encoder_log_reg_injury.pkl')),
        'XGBoost': joblib.load(os.path.join(model_dir, 'xgb_injury_label_encoder.pkl'))
    }
}

# 2. load and prepare data with column name standardization
def load_and_prepare_data(filepath):
    df = pd.read_excel(filepath)
    
    # standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    illness_columns = ['type_of_illness', 'illness_severity']
    injury_columns = ['injury_location', 'injury_type']
    df['illness_text'] = df[illness_columns].fillna('').agg(' '.join, axis=1)
    df['injury_text'] = df[injury_columns].fillna('').agg(' '.join, axis=1)
    df['injury_information'] = df['injury_text'].fillna("No Injury")
    df['illness_information'] = df['illness_text'].fillna("No Illness")
    
    # ensure required columns exist
    if 'user_id' not in df.columns:
        df['user_id'] = np.arange(1, len(df)+1)
    
    if 'date' not in df.columns:
        df['date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # convert date to string if it's datetime
    if pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

test_df = load_and_prepare_data("cleaned_test_data_v3.xlsx")
realuse_df = load_and_prepare_data("cleaned_realuse_data_v3.xlsx")

# 3. enhanced prediction function
def predict_with_models(text, problem_type, actual):
    predictions = {}
    if text in ["No Illness", "No Injury"]:
        return {model: ("No Prediction", 0) for model in models[problem_type].keys()}
    
    vectorized = vectorizers[problem_type].transform([text])
    
    for model_name, model in models[problem_type].items():
        pred = model.predict(vectorized)[0]
        
        if model_name in ['Logistic Regression', 'XGBoost']:
            pred = label_encoders[problem_type][model_name].inverse_transform([pred])[0]
        
        similarity = SequenceMatcher(None, str(actual), str(pred)).ratio() * 100
        predictions[model_name] = (pred, similarity)
    
    return predictions

# 4. process dataset to create flattened results - only for rows with illness or injury
def process_dataset(df):
    illness_results = []
    injury_results = []
    
    for _, row in df.iterrows():
        user_id = str(row['user_id'])
        date = str(row['date'])
        
        # process illness if present
        if row['illed'] == 1 and row['illness_information'] != "No Illness":
            illness_actual = row['illness_information']
            illness_preds = predict_with_models(illness_actual, 'illness', illness_actual)
            
            illness_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': illness_actual
            }
            
            for model_name, (predicted, similarity) in illness_preds.items():
                illness_data[f"{model_name} Predicted"] = predicted
                illness_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                illness_data[f"{model_name} Correct"] = illness_actual == predicted if predicted != "No Prediction" else False
            
            illness_results.append(illness_data)
        
        # process injury if present
        if row['injured'] == 1 and row['injury_information'] != "No Injury":
            injury_actual = row['injury_information']
            injury_preds = predict_with_models(injury_actual, 'injury', injury_actual)
            
            injury_data = {
                'User ID': user_id,
                'Date': date,
                'Actual': injury_actual
            }
            
            for model_name, (predicted, similarity) in injury_preds.items():
                injury_data[f"{model_name} Predicted"] = predicted
                injury_data[f"{model_name} Accuracy (%)"] = round(similarity, 1)
                injury_data[f"{model_name} Correct"] = injury_actual == predicted if predicted != "No Prediction" else False
            
            injury_results.append(injury_data)
            
    def sort_dataframe(df):
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(['Date', 'User ID'])
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df
    
    return {
        'illness': sort_dataframe(pd.DataFrame(illness_results)),
        'injury': sort_dataframe(pd.DataFrame(injury_results))
    }

# 5. process both datasets
test_results = {
    'illness': process_dataset(test_df)['illness'],
    'injury': process_dataset(test_df)['injury']
}

realuse_results = {
    'illness': process_dataset(realuse_df)['illness'],
    'injury': process_dataset(realuse_df)['injury']
}

# 6. create Excel report with enhanced formatting
def create_excel_report():
    output_file = 'additional_report_v3_no_severity.xlsx'
    print(f"\nGenerating Excel report: {output_file}")
    
    # create styles
    green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
    red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
    header_fill = PatternFill(start_color='4472C4', end_color='4472C4', fill_type='solid')
    header_font = Font(color='FFFFFF', bold=True)
    summary_header_fill = PatternFill(start_color='7030A0', end_color='7030A0', fill_type='solid')
    center_alignment = Alignment(horizontal='center')
    
    color_scale_rule = ColorScaleRule(
        start_type='num', start_value=0, start_color='FF0000',
        mid_type='num', mid_value=50, mid_color='FFFF00',
        end_type='num', end_value=100, end_color='00FF00'
    )

    try:
        # create a new workbook
        wb = openpyxl.Workbook()
        
        # create sheets for each dataset and problem type
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                sheet_name = f"{dataset_name} {problem_type.capitalize()}"[:31]
                ws = wb.create_sheet(sheet_name)
                
                # write the data to the worksheet
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
                    for c_idx, value in enumerate(row, 1):
                        ws.cell(row=r_idx, column=c_idx, value=value)
                
                # apply header formatting
                for cell in ws[1]:
                    cell.fill = header_fill
                    cell.font = header_font
                    cell.alignment = center_alignment
                
                # find accuracy and correct columns
                for col_idx, col in enumerate(ws.iter_cols(), 1):
                    col_name = col[0].value
                    
                    if col_name and 'Accuracy (%)' in col_name:
                        # apply color scale to accuracy columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            color_scale_rule
                        )
                    
                    elif col_name and 'Correct' in col_name:
                        # apply green/red to correctness columns
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['TRUE'], fill=green_fill)
                        )
                        ws.conditional_formatting.add(
                            f"{openpyxl.utils.get_column_letter(col_idx)}2:{openpyxl.utils.get_column_letter(col_idx)}{ws.max_row}",
                            CellIsRule(operator='equal', formula=['FALSE'], fill=red_fill)
                        )
                
                # auto-size columns
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(str(cell.value))
                        except:
                            pass
                    adjusted_width = (max_length + 2)
                    ws.column_dimensions[column_letter].width = adjusted_width
        
        # create summary sheet
        ws_summary = wb.create_sheet("Model Performance Summary")
        
        # prepare summary data
        summary_data = []
        models_list = ['Random Forest', 'Logistic Regression', 'XGBoost']
        
        for dataset_name, results in [('Test Data', test_results), ('Real-use Data', realuse_results)]:
            for problem_type in ['illness', 'injury']:
                df = results[problem_type]
                if len(df) == 0:
                    continue
                
                for model in models_list:
                    # calculate average accuracy
                    acc_col = f"{model} Accuracy (%)"
                    if acc_col in df.columns:
                        avg_accuracy = df[acc_col].mean()
                        
                        # calculate error rate (false predictions percentage)
                        correct_col = f"{model} Correct"
                        if correct_col in df.columns:
                            total_predictions = len(df[correct_col])
                            true_predictions = len(df[df[correct_col] == True])
                            accuracy_rate = (true_predictions / total_predictions) * 100 if total_predictions > 0 else 0
                            
                            summary_data.append({
                                'Dataset': dataset_name,
                                'Problem Type': problem_type.capitalize(),
                                'Model': model,
                                'Average Accuracy (%)': round(avg_accuracy, 1),
                                'Accuracy Rate (%)': round(accuracy_rate, 1),
                                'Total Predictions': total_predictions
                            })
        
        # create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        
        # write summary to Excel
        for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws_summary.cell(row=r_idx, column=c_idx, value=value)
        
        # format summary sheet
        for cell in ws_summary[1]:
            cell.fill = summary_header_fill
            cell.font = header_font
            cell.alignment = center_alignment
        
        # apply conditional formatting to accuracy and error rate
        ws_summary.conditional_formatting.add(
            f"D2:D{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000',
                mid_type='num', mid_value=50, mid_color='FFFF00',
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        ws_summary.conditional_formatting.add(
            f"E2:E{ws_summary.max_row}",
            ColorScaleRule(
                start_type='num', start_value=0, start_color='FF0000', 
                mid_type='num', mid_value=50, mid_color='FFFF00', 
                end_type='num', end_value=100, end_color='00FF00'
            )
        )
        
        # Auto-size columns
        for column in ws_summary.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2)
            ws_summary.column_dimensions[column_letter].width = adjusted_width
        
        # Add charts to summary sheet
        chart1 = BarChart()
        chart1.title = "Average Accuracy by Model"
        chart1.y_axis.title = "Accuracy (%)"
        chart1.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=4, min_row=1, max_row=ws_summary.max_row, max_col=4)
        cats = Reference(ws_summary, min_col=3, min_row=2, max_row=ws_summary.max_row)
        chart1.add_data(data, titles_from_data=True)
        chart1.set_categories(cats)
        ws_summary.add_chart(chart1, "G2")
        
        chart2 = BarChart()
        chart2.title = "Error Rate by Model"
        chart2.y_axis.title = "Error Rate (%)"
        chart2.x_axis.title = "Model"
        
        data = Reference(ws_summary, min_col=5, min_row=1, max_row=ws_summary.max_row, max_col=5)
        chart2.add_data(data, titles_from_data=True)
        chart2.set_categories(cats)
        ws_summary.add_chart(chart2, "G20")
        
        # Remove default sheet if it exists
        if 'Sheet' in wb.sheetnames:
            wb.remove(wb['Sheet'])
        
        # Save the workbook
        wb.save(output_file)
        print(f"Successfully saved report to: {os.path.abspath(output_file)}")
        return output_file
    
    except Exception as e:
        print(f"Error generating Excel report: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 7. generate the report
report_file = create_excel_report()



Generating Excel report: additional_report_v3_no_severity.xlsx
Successfully saved report to: /Users/dingyangzuo/additional_report_v3_no_severity.xlsx
